## ML Final Exam



## Question 2 Clustering Task (15 Marks):
The "Urban Mobility Patterns" dataset provides insights into the daily commuting patterns of
individuals in a metropolitan city. The dataset consists of 5,000 entries, 4 features (Average_Speed,
Waiting_Time, Daily_Commute_Distance, Traffic_Congestion_Score) each representing an
individual's daily mobility metrics. The objective is to cluster individuals based on these metrics to
identify distinct mobility patterns and provide insights into urban transportation challenges.
Perform the following tasks:


In [ ]:
# %pip install minisom
# %pip install scikit-learn==0.23.2
# %pip install sompy

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.cluster import KMeans, DBSCAN, MeanShift, AgglomerativeClustering, OPTICS
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import ParameterGrid
from sklearn.exceptions import ConvergenceWarning

from minisom import MiniSom
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

### a.) Data Preprocessing


In [ ]:
# Load the data to the DataFrame
df_ML_q2 = pd.read_excel('Urban_Mobility_Patterns_Data.xlsx')
# df_ML_q2 = pd.read_csv('Urban_Mobility_Patterns_Data.csv)

print(df_ML_q2.shape)
df_ML_q2.head()

In [ ]:
df_ML_q2.describe()

#df_ML_q2.dtypes

In [ ]:
# Check for missing values in the dataset
df_ML_q2.isnull().sum()

In [ ]:
# Verify the nature of missing values; 
# if the rows with missing value in 'Average_Speed' have missing values in 'Daily_Commute_Distance' as well.

# Check if missing values in 'Average_Speed' correspond to missing values in 'Daily_Commute_Distance'
missing_Average_Speed = df_ML_q2['Average_Speed'].isnull()
missing_Daily_Commute_Distance = df_ML_q2['Daily_Commute_Distance'].isnull()

# Check if all rows (index) with missing 'Average_Speed' also have missing 'Daily_Commute_Distance'
all_missing_average_speed_have_missing_daily_commute_distance = missing_Average_Speed.equals(missing_Daily_Commute_Distance)

# Print the result
print(f"All rows with missing 'Average_Speed' also missing 'Daily_Commute_Distance': {all_missing_average_speed_have_missing_daily_commute_distance}")


### Use LinearRegression as the Imputation Method

In [ ]:
# Impute the missing values


# Columns to impute
columns_to_impute = ['Average_Speed', 'Daily_Commute_Distance']

# Create a copy of the DataFrame for imputation
df_to_impute = df_ML_q2.copy()

# Initialize variables to keep track of the best imputation
best_imputation = None
smallest_nulls = float('inf')  # Initialize with a large value

# Loop through each column to impute
for column in columns_to_impute:
    # Identify features (independent variables) and target (variable to be imputed)
    features = df_to_impute.drop(columns=columns_to_impute + ['Traffic_Congestion_Score', column])
    target = df_to_impute[column]

    # Identify missing values
    missing_values = target.isnull()

    # Split the dataset into known and unknown values
    features_known = features[~missing_values]
    features_unknown = features[missing_values]
    target_known = target[~missing_values]

    # Initialize a linear regression model
    regression_model = LinearRegression()

    # Use GridSearchCV to find the best parameters for imputation
    param_grid = {'fit_intercept': [True, False]}
    grid_search = GridSearchCV(regression_model, param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(features_known, target_known)

    # Get the best model from GridSearchCV
    best_model = grid_search.best_estimator_

    # Predict the unknown values
    imputed_values = best_model.predict(features_unknown)

    # Update the DataFrame with imputed values
    df_to_impute.loc[missing_values, column] = imputed_values

    # Check the number of remaining null values
    nulls = df_to_impute[columns_to_impute].isnull().sum().sum()

    # Update the best imputation if the current one has fewer nulls
    if nulls < smallest_nulls:
        best_imputation = df_to_impute.copy()
        smallest_nulls = nulls

# Print the imputed DataFrame
LR_grid_imputated = best_imputation
print("Imputed DataFrame:")
print(LR_grid_imputated)
# Verify the imputed DataFrame
print(LR_grid_imputated.isnull().sum())
print(LR_grid_imputated.describe())

### Setup for comparision between with GridSearch and Without

In [ ]:
df_ML_q2 = LR_grid_imputated

In [ ]:
# To see the distribution of the features in the dataset, use the pairplot() function from the seaborn library.
sns.pairplot(df_ML_q2, diag_kind = "kde")

In [ ]:
# Extract features and target variable
X_q2 = df_ML_q2[['Average_Speed', 'Waiting_Time', 'Daily_Commute_Distance']]
y_q2 = df_ML_q2['Traffic_Congestion_Score']

feature_names = df_ML_q2.columns

### Normalize the features of the dataset.

In [ ]:
scaler = StandardScaler()
X_q2_normalized = scaler.fit_transform(X_q2)
# Check for NaN values in the normalized data

if np.isnan(X_q2_normalized).any():
    print("The normalized data contains NaN values. Please handle missing values.")
else:
    print("The normalized data does not contain NaN values. Can continue with clustering.")

Checking for NaN values after normalizing features is a good practice for several reasons:

Identification of Issues: NaN values in the data may indicate issues with the data preprocessing or feature engineering steps. It could be a result of missing values in the original dataset, and normalizing features with missing values might lead to unexpected or incorrect results.

Impact on Models: Many machine learning algorithms, including clustering algorithms, may not handle NaN values well. The presence of NaN values in the data could cause errors during the training or evaluation of the models. By checking for NaN values, you can ensure that your data is in a suitable format for model training.

Debugging: If NaN values are present, it helps in identifying where in the dataset they occur. This information is valuable for debugging and fixing the underlying issues in data preprocessing.

Consistency: It ensures consistency in handling missing values throughout your data preprocessing pipeline. If NaN values are not handled properly during normalization, it might lead to unexpected behavior downstream in the analysis.

### b.) Model Building (5 marks)

Several machine learning models are commonly used for clustering, which is an unsupervised learning task aimed at grouping similar data points together. Here are some popular clustering algorithms:

1) K-Means Clustering: Divides data into 'k' clusters based on centroids.
    Pros: Simple, easy to understand, and computationally efficient.
    Cons: Sensitive to initial centroids, assumes clusters are spherical.

2) Hierarchical Clustering: Builds a hierarchy of clusters, either top-down (divisive) or bottom-up (agglomerative).
    Pros: Provides a tree-like structure of clusters.
    Cons: Computationally expensive for large datasets.

3) DBSCAN (Density-Based Spatial Clustering of Applications with Noise): Groups together data points that are close to each other and have a sufficient number of neighbors.
    Pros: Can find clusters of arbitrary shapes, robust to outliers.
    Cons: Sensitive to parameters.

4) Mean Shift: Aims to find dense regions in the data by iteratively shifting points towards the mode (peak) of the data distribution.
    Pros: No need to specify the number of clusters, works well with various shapes.
    Cons: Computationally expensive.

5) Gaussian Mixture Model (GMM): Assumes that the data is generated from a mixture of several Gaussian distributions.
    Pros: Provides probabilities of belonging to each cluster, flexible in terms of cluster covariance.
    Cons: Sensitive to initialization.

6) OPTICS (Ordering Points To Identify the Clustering Structure): An extension of DBSCAN that can discover clusters with varying densities.
    Pros: Robust to noise and outliers.
    Cons: Sensitive to parameters.

7) Self-Organizing Maps (SOM): Neural network-based method that reduces dimensionality and projects data onto a 2D grid.
    Pros: Can capture non-linear relationships, useful for visualization.
    Cons: May require tuning of hyperparameters.
    
These algorithms cater to different types of data and structures, and the choice of which one to use often depends on the characteristics of the dataset and the goals of the analysis.

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_q2_normalized, y_q2, test_size=0.2, random_state=42)

# Check the shape of the training and test sets
print("Shape of training set:", X_train.shape)
print("Shape of test set:", X_test.shape)
print("Shape of training labels:", y_train.shape)
print("Shape of test labels:", y_test.shape)

In [ ]:
# model K-Means


# Create empty lists to store results for each model
kmeans_results = []

# 1) K-Means Clustering
param_grid_kmeans = {'n_clusters': range(2, 11), 'n_init': [10]}
best_silhouette_kmeans = float('-inf')
best_num_clusters_kmeans = None
best_labels_kmeans = None
best_mse_kmeans = None
best_mae_kmeans = None
best_r2_kmeans = None

for params_kmeans in ParameterGrid(param_grid_kmeans):
    kmeans = KMeans(**params_kmeans)
    labels_kmeans = kmeans.fit_predict(X_train)
    silhouette_kmeans = silhouette_score(X_train, labels_kmeans)

    # Assuming y_train is your target variable
    y_pred_kmeans = kmeans.predict(X_train)

    # Calculate metrics for training data
    mse_kmeans = mean_squared_error(y_train, y_pred_kmeans)
    mae_kmeans = mean_absolute_error(y_train, y_pred_kmeans)
    r2_kmeans = r2_score(y_train, y_pred_kmeans)

    if silhouette_kmeans > best_silhouette_kmeans:
        best_silhouette_kmeans = silhouette_kmeans
        best_num_clusters_kmeans = len(set(labels_kmeans))
        best_labels_kmeans = labels_kmeans
        best_mse_kmeans = mse_kmeans
        best_mae_kmeans = mae_kmeans
        best_r2_kmeans = r2_kmeans

# Store K-Means results
kmeans_results.append({
    'model_name': 'K-Means Clustering',
    'silhouette_score': best_silhouette_kmeans,
    'num_clusters': best_num_clusters_kmeans,
    'labels': best_labels_kmeans,
    'mse': best_mse_kmeans,
    'mae': best_mae_kmeans,
    'r2': best_r2_kmeans
})

# Print results
print(f"Best Silhouette Score: {best_silhouette_kmeans}")
print(f"Best Number of Clusters: {best_num_clusters_kmeans}")
print(f"Best Mean Squared Error (MSE): {best_mse_kmeans}")
print(f"Best Mean Absolute Error (MAE): {best_mae_kmeans}")
print(f"Best R-squared (R2): {best_r2_kmeans}")

In [ ]:
# Scatter plot


plt.figure(figsize=(13, 5))
sns.scatterplot(x=X_train[:, 0], y=X_train[:, 1], hue=best_labels_kmeans, palette='viridis', legend='full')

# Customize the plot with dynamic feature names and increase font size
plt.title('K-Means Clustering Scatter Diagram for X_train', fontsize=16)
plt.xlabel(feature_names[0], fontsize=14)  # Use the first feature name
plt.ylabel(feature_names[3], fontsize=14)  # Use the second feature name    

# Show the legend
plt.legend(title='Cluster Label', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=12)

plt.show()


In [ ]:
# Box Plot over the features


# Average_speed
plt.figure(figsize=(10, 6))
df_temp_kmeans_avg_speed = pd.DataFrame({'Cluster Label': best_labels_kmeans, 'Feature': X_train[:, 0]})
sns.boxplot(x='Cluster Label', y='Feature', data=df_temp_kmeans_avg_speed, palette='viridis')
plt.title('K-Means Clustering Box Plot for X_train - Average_Speed')
plt.xlabel('Cluster Label')  
plt.ylabel(feature_names[0])  # 0 = avg_speed
plt.show()

# Waiting time
plt.figure(figsize=(10, 6))
df_temp_kmeans_waiting_time = pd.DataFrame({'Cluster Label': best_labels_kmeans, 'Feature': X_train[:, 1]})
sns.boxplot(x='Cluster Label', y='Feature', data=df_temp_kmeans_waiting_time, palette='viridis')
plt.title('K-Means Clustering Box Plot for X_train - Waiting_Time')
plt.xlabel('Cluster Label')  
plt.ylabel(feature_names[1])  # 1 = waiting time
plt.show()

# Daily commute distance
plt.figure(figsize=(10, 6))
df_temp_kmeans_daily_commute = pd.DataFrame({'Cluster Label': best_labels_kmeans, 'Feature': X_train[:, 2]})
sns.boxplot(x='Cluster Label', y='Feature', data=df_temp_kmeans_daily_commute, palette='viridis')
plt.title('K-Means Clustering Box Plot for X_train - Daily_Commute_Distance')
plt.xlabel('Cluster Label')
plt.ylabel(feature_names[2])  # 2 = daily commute distance
plt.show()

Insights from the box plots:
The normalization process scales features to a common range. This allows for a fair comparison between features with different units and magnitudes. We recognize a pattern in 2 major groups regarding avg_speed. One with higher average and one with lower than average. Clusters 0, 5 and 8 indicating more variability. Cluster 3 seems to have many outliers, also in daily commute distance. Regarding waiting time cluster 3 and 7 has the the largest spread.
The interquartile range in cluster 3 is the biggest in any of the plots. Clusters illustrate differences in commute distances, people in different clusters tend to have varying commute distances. The spread of commute distances vary between clusters and Cluster 3 seems to have many outliers. Most of the clusters look quite similar in size. Since cluster 3 has many outliers it's hard to compare with the other clusters.



In [ ]:
# Cluster Distribution:


# Plot Cluster Distribution
sns.countplot(x='labels', data=pd.DataFrame({'labels': best_labels_kmeans}))
plt.title('Cluster Distribution')
plt.xlabel('Cluster Label')
plt.ylabel('Count')
plt.show()

In [ ]:
# 2) Hierarchical Clustering model


# Create empty lists to store results for each model
hierarchical_results = []

param_grid_hierarchical = {'n_clusters': range(2, 11)}
best_silhouette_hierarchical = float('-inf')
best_num_clusters_hierarchical = None
best_labels_hierarchical = None
best_mse_hierarchical = None
best_mae_hierarchical = None
best_r2_hierarchical = None

for params_hierarchical in ParameterGrid(param_grid_hierarchical):
    hierarchical = AgglomerativeClustering(**params_hierarchical)
    labels_hierarchical = hierarchical.fit_predict(X_train)
    silhouette_hierarchical = silhouette_score(X_train, labels_hierarchical)

    # Assuming y_train is your target variable
    y_pred_hierarchical = hierarchical.fit_predict(X_train)

    # Calculate metrics for training data
    mse_hierarchical = mean_squared_error(y_train, y_pred_hierarchical)
    mae_hierarchical = mean_absolute_error(y_train, y_pred_hierarchical)
    r2_hierarchical = r2_score(y_train, y_pred_hierarchical)

    if silhouette_hierarchical > best_silhouette_hierarchical:
        best_silhouette_hierarchical = silhouette_hierarchical
        best_num_clusters_hierarchical = len(set(labels_hierarchical))
        best_labels_hierarchical = labels_hierarchical
        best_mse_hierarchical = mse_hierarchical
        best_mae_hierarchical = mae_hierarchical
        best_r2_hierarchical = r2_hierarchical

# Store Hierarchical results
hierarchical_results.append({
    'model_name': 'Hierarchical Clustering',
    'silhouette_score': best_silhouette_hierarchical,
    'num_clusters': best_num_clusters_hierarchical,
    'labels': best_labels_hierarchical,
    'mse': best_mse_hierarchical,
    'mae': best_mae_hierarchical,
    'r2': best_r2_hierarchical
})

# Print results
print(f"Best Silhouette Score for Hierarchical Clustering: {best_silhouette_hierarchical}")
print(f"Best Number of Clusters for Hierarchical Clustering: {best_num_clusters_hierarchical}")
print(f"Best Mean Squared Error (MSE) for Hierarchical Clustering: {best_mse_hierarchical}")
print(f"Best Mean Absolute Error (MAE) for Hierarchical Clustering: {best_mae_hierarchical}")
print(f"Best R-squared (R2) for Hierarchical Clustering: {best_r2_hierarchical}")

In [ ]:
# box plot - Hierarchical Clustering


df_temp = pd.DataFrame(X_train, columns=['Average_Speed', 'Waiting_Time', 'Daily_Commute_Distance'])

df_temp['Hierarchical_Cluster'] = best_labels_hierarchical

# Plot box plots for each feature across Hierarchical Clusters
plt.figure(figsize=(16, 8))
sns.set_theme(style="whitegrid")
sns.boxplot(data=df_temp.melt(id_vars='Hierarchical_Cluster'), x='Hierarchical_Cluster', y='value', hue='variable', showfliers=False)
plt.title('Box Plots for Features Across Hierarchical Clusters')
plt.show()

In [ ]:
# Plot the scatter diagram with dynamic feature names for X_train


# Choose the best result from Hierarchical Clustering
best_hierarchical_result = hierarchical_results[0]

# Plot the scatter diagram for Hierarchical Clustering
plt.figure(figsize=(13, 5))
sns.scatterplot(x=X_train[:, 0], y=X_train[:, 1], hue=best_hierarchical_result['labels'], palette='viridis', legend='full')
plt.title('Hierarchical Clustering Scatter Diagram for X_train')
plt.xlabel(feature_names[0])  # Use the first feature name
plt.ylabel(feature_names[1])  # Use the second feature name

# Add information about the model and evaluation metrics
info_text = f"Model: {best_hierarchical_result['model_name']}\n"
info_text += f"Number of Clusters: {best_hierarchical_result['num_clusters']}\n"
info_text += f"Silhouette Score: {best_hierarchical_result['silhouette_score']:.2f}\n"
info_text += f"Mean Squared Error (MSE): {best_hierarchical_result['mse']:.2f}\n"
info_text += f"Mean Absolute Error (MAE): {best_hierarchical_result['mae']:.2f}\n"
info_text += f"R-squared (R2): {best_hierarchical_result['r2']:.2f}"

# Show the legend
plt.legend(title='Cluster Label', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# 3) DBSCAN Clustering


# Create empty lists to store results for each model
dbscan_results = []

# Define a range of parameters to search for DBSCAN
param_grid_dbscan = {'eps': [0.1, 0.5, 1.0, 1.5], 'min_samples': [3, 5, 10, 15, 20]}
cluster_range = range(2, 11)  # Range of cluster numbers to search for

best_silhouette_dbscan = float('-inf')
best_params_dbscan = {}
best_labels_dbscan = None

for params_dbscan in ParameterGrid(param_grid_dbscan):
    for n_clusters in cluster_range:
        dbscan = DBSCAN(**params_dbscan)
        labels_dbscan = dbscan.fit_predict(X_q2_normalized)

        # Check if the number of clusters found by DBSCAN is equal to the current iteration's cluster number
        if len(set(labels_dbscan)) == n_clusters:
            silhouette_dbscan = silhouette_score(X_q2_normalized, labels_dbscan)

            if silhouette_dbscan > best_silhouette_dbscan:
                best_silhouette_dbscan = silhouette_dbscan
                best_params_dbscan = {**params_dbscan, 'n_clusters': n_clusters}
                best_labels_dbscan = labels_dbscan

# Append results to the list
dbscan_results.append({
    'model_name': 'DBSCAN Clustering',
    'silhouette_score': best_silhouette_dbscan,
    'num_clusters': best_params_dbscan['n_clusters'],
    'params': best_params_dbscan
})

# Print the results
for results in dbscan_results:
    print(f"{results['model_name']}: Silhouette Score - {results['silhouette_score']}, \nNumber of Clusters - {results['num_clusters']}, Parameters - {results['params']}")

In [ ]:
# box plot - DBSCAN


df_temp_dbscan = pd.DataFrame(X_q2, columns=['Average_Speed', 'Waiting_Time', 'Daily_Commute_Distance'])  # Adjust based on your data

# Assuming best_labels_dbscan contains the optimal labels from DBSCAN Clustering
best_labels_dbscan = best_labels_dbscan  # Use the actual variable name

# Add the 'DBSCAN_Cluster' column to the temporary DataFrame using the best labels
df_temp_dbscan['DBSCAN_Cluster'] = best_labels_dbscan

# Plot box plots for each feature across DBSCAN Clusters
plt.figure(figsize=(16, 8))
sns.set_theme(style="whitegrid")
sns.boxplot(data=df_temp_dbscan.melt(id_vars='DBSCAN_Cluster'), x='DBSCAN_Cluster', y='value', hue='variable', showfliers=False)
plt.title('Box Plots for Features Across DBSCAN Clusters')
plt.show()

The output shows that DBSCAN is consistently finding 6 clusters, but the silhouette score varies slightly. The silhouette score is a measure of how similar an object is to its own cluster compared to other clusters. It ranges from -1 to 1, where a high value indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters.

In your case, the slight variation in silhouette scores might be due to the sensitivity of the silhouette score to the local structure of the data. Different runs of the algorithm may result in slightly different assignments of points to clusters, leading to small variations in silhouette scores.

Additionally, the silhouette score itself is not always a definitive measure, and its interpretation can depend on the nature of the data. It's often a good practice to consider other evaluation metrics and, in some cases, to visually inspect the clustering results to make a more informed assessment.

If you're consistently getting 6 clusters with similar silhouette scores, it indicates a stable clustering structure for your data with DBSCAN. However, if you're looking for more consistency, you might want to explore other clustering algorithms or fine-tune the parameters further to see if you can achieve a more stable result.

In [ ]:
# Scatter Plot - DBSCAN Clustering


# Choose the best result from DBSCAN Clustering
best_dbscan_result = dbscan_results[0]

# Plot the scatter diagram for DBSCAN Clustering
plt.figure(figsize=(13, 5))
sns.scatterplot(x=X_q2_normalized[:, 0], y=X_q2_normalized[:, 1], hue=best_labels_dbscan, palette='viridis', legend='full')
plt.title('DBSCAN Clustering Scatter Diagram')
plt.xlabel(feature_names[0]) 
plt.ylabel(feature_names[1]) 

# Add information about the model and evaluation metrics
info_text = f"Model: {best_dbscan_result['model_name']}\n"
info_text += f"Number of Clusters: {best_dbscan_result['num_clusters']}\n"
info_text += f"Silhouette Score: {best_dbscan_result['silhouette_score']:.2f}\n"
info_text += f"Parameters: {best_dbscan_result['params']}"

# Show the legend
plt.legend(title='Cluster Label', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# 4) Mean Shift Clustering Model. This can take up to 3 minutes to run


# Create empty lists to store results for each model
mean_shift_results = []

# Function to find the best Mean Shift Clustering model
def find_best_mean_shift(X, param_grid):
    best_model = None
    best_silhouette = float('-inf')
    best_params = None

    for params in ParameterGrid(param_grid):
        mean_shift = MeanShift(**params)
        labels = mean_shift.fit_predict(X)

        # Check if the number of clusters found by Mean Shift is greater than 1
        if len(set(labels)) > 1:
            silhouette = silhouette_score(X, labels)

            # Update the best model if the silhouette score is higher
            if silhouette > best_silhouette:
                best_silhouette = silhouette
                best_model = mean_shift
                best_params = params

    return best_model, best_silhouette, best_params

# Define the parameter grid for Mean Shift Clustering
param_grid_mean_shift = {'bandwidth': [0.1, 0.5, 1.0, 1.5]}

# Find the best Mean Shift Clustering model
best_mean_shift_model, best_silhouette_mean_shift, best_params_mean_shift = find_best_mean_shift(X_q2_normalized, param_grid_mean_shift)

# Store Mean Shift results
mean_shift_results.append({
    'model_name': 'Mean Shift Clustering',
    'silhouette_score': best_silhouette_mean_shift,
    'num_clusters': len(set(best_mean_shift_model.labels_)),
    'params': best_params_mean_shift
    })

# Print the results
print("Best Mean Shift Clustering with Silhouette Score:", best_silhouette_mean_shift)
print("Best Parameters for Mean Shift Clustering:", best_params_mean_shift)

# Output the number of clusters
num_clusters_mean_shift = len(set(best_mean_shift_model.labels_))
print("Number of Clusters:", num_clusters_mean_shift)

In [ ]:
# box plot - Mean Shift Clustering Model 


# Assuming X_q2 is your original feature matrix with 3 columns
df_temp_mean_shift = pd.DataFrame(X_q2, columns=['Average_Speed', 'Waiting_Time', 'Daily_Commute_Distance'])  # Adjust based on your data

# Assuming best_mean_shift_model.labels_ contains the optimal labels from Mean Shift Clustering
best_labels_mean_shift = best_mean_shift_model.labels_  # Use the actual variable name

# Add the 'Mean_Shift_Cluster' column to the temporary DataFrame using the best labels
df_temp_mean_shift['Mean_Shift_Cluster'] = best_labels_mean_shift

# Plot box plots for each feature across Mean Shift Clusters
plt.figure(figsize=(16, 8))
sns.set_theme(style="whitegrid")
sns.boxplot(data=df_temp_mean_shift.melt(id_vars='Mean_Shift_Cluster'), x='Mean_Shift_Cluster', y='value', hue='variable', showfliers=False)
plt.title('Box Plots for Features Across Mean Shift Clusters')
plt.show()

In [ ]:
# Plot the scatter diagram for Mean Shift Clustering with different colors for each cluster


plt.figure(figsize=(12, 6))
sns.scatterplot(x=X_q2_normalized[:, 0], y=X_q2_normalized[:, 1], hue=best_mean_shift_model.labels_, palette='muted',legend='full', marker='o')
plt.title('Mean Shift Clustering Scatter Diagram')
plt.xlabel(feature_names[0])
plt.ylabel(feature_names[1])
plt.show()

In [ ]:
# 5) Gaussian Mixture Model (GMM)  


# Create empty lists to store results for each model
gmm_results = []

# Function to find the best Gaussian Mixture Model
def find_best_gmm(X, param_grid):
    best_model = None
    best_silhouette = float('-inf')
    best_params = None

    for params in ParameterGrid(param_grid):
        gmm = GaussianMixture(**params)
        labels = gmm.fit_predict(X)

        # Check if the number of clusters found by GMM is greater than 1
        if len(set(labels)) > 1:
            silhouette = silhouette_score(X, labels)

            # Update the best model if the silhouette score is higher
            if silhouette > best_silhouette:
                best_silhouette = silhouette
                best_model = gmm
                best_params = params

    return best_model, best_silhouette, best_params

# Disable ConvergenceWarnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Define the parameter grid for Gaussian Mixture Model
param_grid_gmm = {'n_components': range(2, 11)}

# Find the best Gaussian Mixture Model
best_gmm_model, best_silhouette_gmm, best_params_gmm = find_best_gmm(X_q2_normalized, param_grid_gmm)

# Re-enable warnings
warnings.filterwarnings("default", category=ConvergenceWarning)

# Store GMM results
gmm_results.append({
    'model_name': 'Gaussian Mixture Model',
    'silhouette_score': best_silhouette_gmm,
    'num_clusters': best_gmm_model.n_components,
    'params': best_params_gmm
})

# Print the results
print("Best Gaussian Mixture Model with Silhouette Score:", best_silhouette_gmm)
print("Best Parameters for Gaussian Mixture Model:", best_params_gmm)
print("Number of Clusters:", best_gmm_model.n_components)

In [ ]:
# box plot for Gaussian Mixture Model (GMM) 


df_temp_gmm = pd.DataFrame(X_q2, columns=['Average_Speed', 'Waiting_Time', 'Daily_Commute_Distance'])

# use the best labels from gmm
best_labels_gmm = best_gmm_model.predict(X_q2_normalized)

# Add the 'GMM_Cluster' column to the temporary DataFrame using the best labels
df_temp_gmm['GMM_Cluster'] = best_labels_gmm

# Plot box plots for each feature across GMM Clusters
plt.figure(figsize=(16, 8))
sns.set_theme(style="whitegrid")
sns.boxplot(data=df_temp_gmm.melt(id_vars='GMM_Cluster'), x='GMM_Cluster', y='value', hue='variable', showfliers=False)
plt.title('Box Plots for Features Across GMM Clusters')
plt.show()

In [ ]:
# Plot the scatter diagram for the best Gaussian Mixture Model with different colors for each cluster


plt.figure(figsize=(12, 6))
sns.scatterplot(x=X_q2_normalized[:, 0], y=X_q2_normalized[:, 1], hue=best_gmm_model.predict(X_q2_normalized), palette='muted', marker='o')
plt.title('Gaussian Mixture Model Clustering Scatter Diagram')
plt.xlabel(feature_names[0])
plt.ylabel(feature_names[1])
plt.show()

In [ ]:
# 6) OPTICS Clustering


# Create empty lists to store results for each model
optics_results = []

# Define a range of parameters to search for OPTICS
param_grid_optics = {'min_samples': [3, 5, 7, 10], 'xi': [0.01, 0.05, 0.1, 0.2]}
best_silhouette_optics = float('-inf')
best_params_optics = {}

for params_optics in ParameterGrid(param_grid_optics):
    optics = OPTICS(**params_optics)
    labels_optics = optics.fit_predict(X_q2_normalized)

    # Check if OPTICS found more than one cluster
    unique_labels_optics = set(labels_optics)
    if len(unique_labels_optics) > 1:
        silhouette_optics = silhouette_score(X_q2_normalized, labels_optics)

        if silhouette_optics > best_silhouette_optics:
            best_silhouette_optics = silhouette_optics
            best_params_optics = params_optics

# Store OPTICS results
optics_results = [{
    'model_name': 'OPTICS Clustering1',
    'silhouette_score': best_silhouette_optics,
    'num_clusters': len(unique_labels_optics),
    'params': best_params_optics
}]

# Print the results
for results in optics_results:
    params_str = results.get('params', {}) 
    print(f"{results['model_name']}: Silhouette Score - {results['silhouette_score']}, Number of Clusters - {results['num_clusters']}, Parameters - {params_str}")
    print(f"Number of Clusters: {results['num_clusters']}")

# Plot the OPTICS clustering scatter diagram with 'muted' colormap
plt.figure(figsize=(12, 6))
palette = sns.color_palette("muted", n_colors=len(unique_labels_optics))
sns.scatterplot(x=X_q2_normalized[:, 0], y=X_q2_normalized[:, 1], hue=labels_optics, palette=palette, marker='o')

plt.title('OPTICS Clustering Scatter Diagram')
plt.xlabel(feature_names[0])
plt.ylabel(feature_names[1])
plt.legend(title='Cluster Label')
plt.show()

The OPTICS clustering algorithm is designed to discover density-based clusters in data. The silhouette score is a measure of how well-defined the clusters are, with a higher silhouette score indicating better-defined clusters. In your case, the silhouette score is negative, and the number of clusters is very high (696). This suggests that OPTICS is not performing well on your data, and the resulting clusters may not be meaningful.

In [ ]:
# box plot for optics


df_temp_optics = pd.DataFrame(X_q2, columns=['Average_Speed', 'Waiting_Time', 'Daily_Commute_Distance'])  # Adjust based on your data

# Best_labels_optics contains the optimal labels from OPTICS Clustering
best_labels_optics = labels_optics

# Add the 'OPTICS_Cluster' column to the temporary DataFrame using the best labels
df_temp_optics['OPTICS_Cluster'] = best_labels_optics

# Plot box plots for each feature across OPTICS Clusters
plt.figure(figsize=(16, 8))
sns.set_theme(style="whitegrid")
sns.boxplot(data=df_temp_optics.melt(id_vars='OPTICS_Cluster'), x='OPTICS_Cluster', y='value', hue='variable', showfliers=False)
plt.title('Box Plots for Features Across OPTICS Clusters')
plt.show()

OPTICS (Ordering Points To Identify the Clustering Structure) is generally not very sensitive to the scale of the features, so it can work on both normalized and unnormalized data. Normalization, which scales features to a standard range, is often recommended for many clustering algorithms to ensure that all features contribute equally to the distance calculations.

However, OPTICS has an inherent ability to handle data with varying densities, making it less sensitive to the scale of features compared to some other clustering algorithms. Therefore, while normalization might still be beneficial in some cases, OPTICS may perform reasonably well on unnormalized data.

As a good practice, it's recommended to try both normalized and unnormalized data and observe the performance to determine the most suitable approach for your specific dataset. Keep in mind that the optimal preprocessing steps can vary depending on the characteristics of the data and the specific requirements of your clustering task.

In [ ]:
# Optimizing the OPTICS Clustering Model

# 7) Self-Organizing Maps (SOM) Optimization

# Create empty lists to store results for each model
som_results = []

# Define the parameter grid for SOM
param_grid_som = {
    'x': [5, 10, 15],
    'y': [5, 10, 15],
    'input_len': [X_q2_normalized.shape[1]],
    'sigma': [0.5, 1.0, 1.5],
    'learning_rate': [0.1, 0.5, 1.0]
}

best_silhouette_som = float('-inf')
best_params_som = {}

# Perform grid search
for params_som in ParameterGrid(param_grid_som):
    som = MiniSom(**params_som)
    som.train_random(X_q2_normalized, 100)
    labels_som = np.array([som.winner(x) for x in X_q2_normalized]).T[0]
    silhouette_som = silhouette_score(X_q2_normalized, labels_som)

    # Update the best results if the silhouette score is higher
    if silhouette_som > best_silhouette_som:
        best_silhouette_som = silhouette_som
        best_params_som = params_som

# Store SOM results
som_results.append({
    'model_name': 'Self-Organizing Maps Optimized',
    'silhouette_score': best_silhouette_som,
    'num_clusters': len(set(labels_som)),
    'params': best_params_som
})

# Print the optimized results
print(f"Optimized SOM Model: Silhouette Score - {best_silhouette_som}")
print(f"Number of Clusters - {len(set(labels_som))}")
print(f"Model Parameters - {best_params_som}")

In [ ]:
# Create empty lists to store results for each model
som_results = []

# Parameters for MiniSom
som = MiniSom(x=10, y=10, input_len=X_q2_normalized.shape[1], sigma=1.0, learning_rate=0.5)
som.train_random(X_q2_normalized, 100)  # Adjust the number of iterations as needed
labels_som = np.array([som.winner(x) for x in X_q2_normalized]).T[0]

# Calculate silhouette score for SOM
silhouette_som = silhouette_score(X_q2_normalized, labels_som)

# Store SOM results
som_results.append({
    'model_name': 'Self-Organizing Maps01',
    'silhouette_score': silhouette_som,
    'num_clusters': len(set(labels_som)),
})

# Print the optimized results
print(f"Optimized SOM Model: Silhouette Score - {silhouette_som}")
print(f"Number of Clusters - {len(set(labels_som))}")
print(f"Model Parameters - {som_results[0]['model_name']}")


# Plot the scatter diagram with the 'muted' color palette
plt.figure(figsize=(12, 6))
sns.scatterplot(x=X_q2_normalized[:, 0], y=X_q2_normalized[:, 1], hue=labels_som, palette='muted', marker='o')
plt.title('Self-Organizing Maps Scatter Diagram')
plt.xlabel(feature_names[0])
plt.ylabel(feature_names[1])
plt.show()

In [ ]:
# Combine all results into a single list
all_results = (
    kmeans_results + hierarchical_results + dbscan_results +
    mean_shift_results + gmm_results + optics_results + som_results
)

# Print the results
for results in all_results:
    print(f"{results['model_name']}: Silhouette Score - {results['silhouette_score']}, Number of Clusters - {results['num_clusters']}")


it's quite normal to observe different results, including varying silhouette scores and numbers of clusters, when applying different clustering algorithms to the same dataset. Different clustering algorithms make different assumptions about the structure of the data and have different strengths and weaknesses. The choice of the most appropriate algorithm depends on the characteristics of the data and the goals of your analysis.

Here are some factors that can contribute to these differences:

Algorithm Assumptions: Clustering algorithms have different assumptions about the shape and distribution of clusters. K-Means, for example, assumes spherical clusters, while DBSCAN can find clusters of arbitrary shapes. These assumptions can lead to different interpretations of the underlying structure of the data.

Parameter Sensitivity: Clustering algorithms often have parameters that need to be set, such as the number of clusters for K-Means or the epsilon and min_samples for DBSCAN. The choice of these parameters can significantly impact the results.

Handling of Outliers: Algorithms like DBSCAN can identify outliers as noise points, while others like K-Means may assign outliers to the nearest cluster. This can result in different cluster compositions and silhouette scores.

Topology and Density Considerations: Algorithms like OPTICS can handle clusters with varying shapes and densities, which may lead to different outcomes compared to algorithms with rigid assumptions about cluster geometry.

Feature Scaling: Some algorithms are sensitive to the scale of features, so standardizing or normalizing the data may affect results.

Given these variations, it's common practice to try multiple clustering algorithms and compare their results. Silhouette score and the number of clusters are just two metrics among many that you can use to evaluate the quality of the clustering. It's essential to consider the context of your data and the goals of your analysis when choosing the most appropriate algorithm.

In [ ]:
# Bar grapgh - Silhouette scores the models that does not support mse, mae and r2


# Find the index of the model with the highest silhouette score
max_silhouette_index = max(range(len(all_results)), key=lambda i: all_results[i]['silhouette_score'])

# Convert the results to a DataFrame for easier plotting
results_df = pd.DataFrame(all_results)

# Set up the plot
plt.figure(figsize=(10, 6))
sns.set(style="whitegrid")

# Create a horizontal barplot with the number of clusters
ax = sns.barplot(x='num_clusters', y='model_name', data=results_df, palette="Blues")

# Overlay the silhouette scores on the bars
for index, value in enumerate(results_df['num_clusters']):
    text_color = 'white' if index == max_silhouette_index else 'black'  # Adjust text color for better visibility
    ax.text(value + 0.1, index, f"{results_df['silhouette_score'][index]:.3f}", va='center', color=text_color)

# Change the color of the bar for the model with the highest silhouette score to red
ax.patches[max_silhouette_index].set_facecolor('red')

# Display the silhouette score on the bar with the highest silhouette score
ax.text(results_df['num_clusters'][max_silhouette_index] + 0.1, max_silhouette_index,
        f"{results_df['silhouette_score'][max_silhouette_index]:.3f}", va='center', color='red')

# Customize the plot
plt.title('Number of Clusters and Silhouette Scores for Clustering Models')
plt.xlabel('Number of Clusters')
plt.ylabel('Clustering Model')
plt.tight_layout()
# Show the plot
plt.show()

In [ ]:
# compare the models who support mse, mae and r2


# Set a seaborn style with shades of blue
sns.set_palette("BuPu")

# Example results list (replace this with your actual results for both K-Means and Hierarchical Clustering)
results_list = [
    {'model_name': 'K-Means Clustering', 'mse': 0.1, 'mae': 0.08, 'r2': 0.9},
    {'model_name': 'Hierarchical Clustering', 'mse': 0.15, 'mae': 0.12, 'r2': 0.85},
    # Add more models and their metrics as needed
]

# Extract model names and relevant metrics
model_names = [result['model_name'] for result in results_list]
mse_values = [result['mse'] for result in results_list]
mae_values = [result['mae'] for result in results_list]
r2_values = [result['r2'] for result in results_list]

# Create a horizontal bar chart with reduced separation between top and bottom groups
fig, ax = plt.subplots(figsize=(10, 5))
bar_width = 0.2  # Adjust the width of each bar
separation_top = 0.22  # Adjust the separation for the top three bars
separation_bottom = 0.22  # Adjust the separation for the bottom three bars

index = np.arange(len(model_names))

bar_mse = ax.barh(index - separation_top, mse_values, bar_width, label='Mean Squared Error (MSE)')
bar_mae = ax.barh(index, mae_values, bar_width, label='Mean Absolute Error (MAE)')
bar_r2 = ax.barh(index + separation_bottom, r2_values, bar_width, label='R-squared (R2)')

# Add values on the bars
for i, value in enumerate(mse_values):
    ax.text(value + 0.02, index[i] - separation_top, f'{value:.2f}', va='center', ha='left')


for i, value in enumerate(mae_values):
    ax.text(value + 0.02, index[i], f'{value:.2f}', va='center', ha='left')


for i, value in enumerate(r2_values):
    ax.text(value + 0.02, index[i] + separation_bottom, f'{value:.2f}', va='center', ha='left')

# Add labels, title, and legend
ax.set_yticks(index)
ax.set_yticklabels(model_names)
ax.set_xlabel('Metrics')
ax.set_title('Comparison Across Models')

# Reverse the order of legends to be in the same order as the bars
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], bbox_to_anchor=(0.65, 0.8), loc='upper left')  # Adjust position here

ax.set_xlim(0, 1.0)
# Show the plot
plt.show()
#plt.savefig('mse_mae_and_r2.png') 